In [15]:
import sys
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output, Image
import time

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from simulation import run_simulation, save_results, load_parameters
from utils import calculate_box_counting_dimension

In [2]:
default_params_path = '../data/raw/parameters.json'
default_params = load_parameters(default_params_path)

def run_simulation_and_display(params):
    
    history, state_counts = run_simulation(params)
    
    simulation_id = time.strftime("%Y%m%d-%H%M%S")
    output_dir = os.path.join('..', 'data', 'results')

    peak_time = pd.DataFrame(state_counts)['infected'].idxmax()
    grid_at_peak = history[peak_time]
    box_counting_dim = calculate_box_counting_dimension(grid_at_peak)
    
    save_results(simulation_id, history, state_counts, params, box_counting_dim, output_dir=output_dir)
    
    results_path = os.path.join(output_dir, f"simulation_{simulation_id}")
    
    clear_output(wait=True)
    
    print(f"Simulação concluída com os parâmetros: {params}")
    print(f"Resultados salvos em: {results_path}")

    seir_image_path = os.path.join(results_path, 'seir_curves.png')
    display(Image(filename=seir_image_path))

    animation_path = os.path.join(results_path, 'simulation.gif')
    display(Image(filename=animation_path))
    
    print(f"Dimensão de box-counting no pico da epidemia: {box_counting_dim:.4f}")

In [3]:
style_layout = {'description_width': '0px'}

random_seed_widget = VBox([
    widgets.Label('Semente Aleatória'),
    widgets.IntText(value=default_params['random_seed'], style=style_layout)])

col1 = VBox([
    widgets.Label('Tamanho da Grade'),
    widgets.IntSlider(min=50, max=200, step=10, value=default_params['grid_size'], style=style_layout),
    widgets.Label('Taxa de Infecção'),
    widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=default_params['infection_rate'], style=style_layout),
    widgets.Label('Taxa de Perda de Imunidade'),
    widgets.FloatSlider(min=0.0, max=0.1, step=0.001, value=default_params['immunity_loss_rate'], style=style_layout),
    widgets.Label('Taxa de Movimento'),
    widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=default_params['movement_rate'], style=style_layout),
])

col2 = VBox([
    widgets.Label('Infectados Iniciais'),
    widgets.IntSlider(min=1, max=100, step=1, value=default_params['initial_infected'], style=style_layout),
    widgets.Label('Duração da Doença'),
    widgets.IntSlider(min=1, max=30, step=1, value=default_params['recovery_time'], style=style_layout),
    widgets.Label('Densidade Populacional'),
    widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=default_params['population_density'], style=style_layout),
    widgets.Label('Tempo de Exposição'),
    widgets.IntSlider(min=1, max=30, step=1, value=default_params['exposed_time'], style=style_layout),
])

run_button = widgets.Button(description="Iniciar Simulação")

simulation_output = widgets.Output()

def on_run_button_clicked(b):
    with simulation_output:
        display(widgets.Label("Simulação em andamento..."))

        params = {
            'grid_size': col1.children[1].value,
            'initial_infected': col2.children[1].value,
            'infection_rate': col1.children[3].value,
            'recovery_time': col2.children[3].value,
            'immunity_loss_rate': col1.children[5].value,
            'population_density': col2.children[5].value,
            'movement_rate': col1.children[7].value,
            'exposed_time': col2.children[7].value,
            'num_steps': default_params['num_steps'],
            'random_seed': random_seed_widget.children[1].value
        }
        run_simulation_and_display(params)

run_button.on_click(on_run_button_clicked)

display(VBox([
    HBox([col1, col2]),
    random_seed_widget,
    run_button,
    simulation_output 
]))